In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop, adam, Adadelta
from keras.utils.np_utils import to_categorical
from keras import optimizers

Using TensorFlow backend.


In [2]:
db = pd.read_csv('C:/Users/samue/Documents/MOOCs/Untitled Folder/train.csv')

#extraction depuis le format dataframe pandas
data = np.array([np.array(db.iloc[i][1:]) for i in range(len(db))])

In [8]:
#Mise à l'échelle des features -> il faut que chaque feature (= position de pixel ici) ait la même moyenne (idéalement 0)
scaler = StandardScaler()
scaler.fit(data)
data_scaled = scaler.transform(data)

In [9]:
X = data_scaled.reshape((42000,28,28,1))
y = np.array(db.label)

oh = OneHotEncoder()
y = oh.fit_transform(y.reshape(-1, 1));

C:\Users\samue\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [76]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [78]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3), strides=1, padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv2D(filters=32, kernel_size=(3,3), strides=1, padding='same'))
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(10))
model.add(Activation("softmax"))

#model.summary()

In [81]:
Adm = optimizers.Adam(0.0005) 

model.compile(optimizer=Adm,
              loss='categorical_crossentropy',metrics = ['accuracy'] )

history1 = model.fit(X_train, Y_train,epochs=1,
          batch_size=20,verbose=1,
          validation_split=0.1)

Train on 30240 samples, validate on 3360 samples
Epoch 1/1
30240/30240 [==============================] - 42s 1ms/step - loss: 0.0139 - acc: 0.9955 - val_loss: 0.0855 - val_acc: 0.9804


In [109]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis = 1)

In [116]:
sum(np.argmax(Y_test.toarray(), axis = 1) == y_pred)/len(y_pred)

0.9802380952380952

In [123]:
db_pred = pd.read_csv('C:/Users/samue/Documents/MOOCs/Untitled Folder/test.csv')
data_pred = np.array([np.array(db_pred.iloc[i]) for i in range(len(db_pred))])

In [124]:
scaler = StandardScaler()
scaler.fit(data_pred)
data_scaled = scaler.transform(data_pred)

In [125]:
X_topred = data_scaled.reshape((28000,28,28,1))

In [128]:
y_pred = model.predict(X_topred)
y_pred = np.argmax(y_pred, axis = 1)

In [132]:
d = {'ImageId': range(1,len(y_pred)+1), 'Label': y_pred}
df = pd.DataFrame(data=d)

df.to_csv("my_submission.csv", index = False)